In [1]:
# import numpy as np
import requests
import psycopg2
import pandas
import json
import boto3
from scipy import spatial

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)


In [2]:
response = requests.get("https://ymh9rihek3.execute-api.sa-east-1.amazonaws.com/prod/LightningFunction")
lightningOccurrences = pandas.read_json(response.text)

In [3]:
PORT=              "3000"
DATABASE_USER=     "pfc"
DATABASE_PASSWORD= "pfctaesa2020**"
DATABASE_HOST=     "pfc.cumxxw3ebrim.sa-east-1.rds.amazonaws.com"
DATABASE_NAME=     "pfc"
DATABASE_PORT=     "5432"

In [4]:
con = psycopg2.connect(host=DATABASE_HOST, database=DATABASE_NAME,user=DATABASE_USER, password=DATABASE_PASSWORD,
                       port=DATABASE_PORT)
cur = con.cursor()
sql = 'SELECT "ds_linha_transmissao", "coord_y", "coord_x" FROM "torres_completas" WHERE "coord_y" IS NOT NULL OR "coord_x" IS NOT NULL'
cur.execute(sql)
results = cur.fetchall()
towers = pandas.DataFrame (results,columns=['linha','latitude','longitude'])

In [5]:
tree = spatial.KDTree(towers[['latitude','longitude']])
distances = tree.query(lightningOccurrences[['latitude','longitude']])
distances = pandas.DataFrame(list(distances)).T
distances.columns = ['distance','towerIndex']
distances["distance"] = distances["distance"]*100000

In [6]:
type([distances["distance"],pandas.DataFrame(lightningOccurrences["corrente"])])

list

In [7]:
type(distances["distance"])

pandas.core.series.Series

In [8]:
trainingDataset = pandas.DataFrame.transpose(pandas.DataFrame([distances["distance"],lightningOccurrences["corrente"]])).to_numpy()

In [9]:
client = boto3.client('sagemaker-runtime')
response = client.invoke_endpoint(EndpointName='sagemaker-scikit-learn-2020-10-07-02-43-19-460',
                                  ContentType="application/json",
                                  Body=json.dumps(trainingDataset.tolist()))

In [10]:
result = response['Body'].read().decode('utf-8')

In [11]:
type(result)

str

In [12]:
teste = list(map(float,result.replace("[", "").replace("]", "").split(', ')))

In [13]:
type(teste)

list

In [14]:
distances["results"] = pandas.DataFrame(teste)

In [15]:
dictPayload = {
    "date": lightningOccurrences["data"].iloc[distances.index[distances['results'] == 1]],
    "towerindex": distances["towerIndex"].iloc[distances.index[distances['results'] == 1]],
    "distance": distances["distance"].iloc[distances.index[distances['results'] == 1]],
    "latitude": lightningOccurrences["latitude"].iloc[distances.index[distances['results'] == 1]],
    "longitude": lightningOccurrences["longitude"].iloc[distances.index[distances['results'] == 1]]
}

In [16]:
finalPayload = json.loads(pandas.DataFrame(dictPayload,columns=['date','towerindex','distance','latitude','longitude']).to_json())


In [17]:
cur.execute("TRUNCATE public.resultados")
cols = finalPayload.keys()
cols_str = ','.join(cols)
vals = [ finalPayload[k] for k in cols ]
vals_str = ','.join( ['%s' for i in range(len(vals))] )
for i in vals[0]:
    rowValues = []
    for j in range(len(vals)):
        rowValues.append(vals[j][str(i)])
    sql_str = """INSERT INTO public.resultados ({}) VALUES ({})""".format(cols_str, vals_str)
    cur.execute(sql_str, rowValues)
con.commit()